# Semantic Kernel

在此代码示例中，您将使用 [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI 框架创建一个基本代理。

本示例的目标是向您展示我们稍后在实现不同代理模式时将在附加代码示例中使用的步骤。


In [1]:
## 导入所需的 Python 包
import os

from typing import Annotated

from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Creating the Client

在此示例中，我们将使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 访问 LLM。

`ai_model_id` 定义为 `gpt-4o-mini`。尝试将模型更改为 GitHub Models 市场中可用的其他模型，以查看不同的结果。

为了使用用于 GitHub Models 的 `base_url` 的 Azure Inference SDK，我们将在 Semantic Kernel 中使用 `OpenAIChatCompletion` 连接器。还有其他 [可用连接器](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) 可用于 Semantic Kernel 与其他模型提供商。


In [3]:
import random

# 为示例定义一个示例插件

class DestinationsPlugin:
    """一个随机度假目的地的列表。"""

    def __init__(self):
        # 度假目的地列表
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # 跟踪上一个目的地以避免重复
        self.last_destination = None

    @kernel_function(description="提供一个随机度假目的地。")
    def get_random_destination(self) -> Annotated[str, "返回一个随机度假目的地。"]:
        # 获取可用目的地（如果可能，排除上一个）
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # 选择一个随机目的地
        destination = random.choice(available_destinations)

        # 更新上一个目的地
        self.last_destination = destination

        return destination

load_dotenv()

client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com/",
)

# 创建一个将由 `ChatCompletionAgent` 使用的 AI 服务
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## 创建代理

下面我们创建一个名为 `TravelAgent` 的代理。

在此示例中，我们使用了非常简单的指令。您可以更改这些指令，看看代理的响应有何不同。


In [4]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="你是一个乐于助人的 AI 代理，可以帮助客户规划随机目的地的假期，回答使用中文。",
)

## 运行代理

现在我们可以通过定义一个类型为 `ChatHistoryAgentThread` 的线程来运行代理。任何所需的系统消息都提供给代理的 invoke_stream `messages` 关键字参数。

定义这些之后，我们创建一个 `user_inputs`，这将是用户发送给代理的消息。在此示例中，我们将此消息设置为 `Plan me a sunny vacation`。

随意更改此消息，看看代理的响应有何不同。


In [6]:
async def main():
    # 为代理创建一个新线程
    # 如果未提供线程，将创建一个新线程，并随初始响应返回
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. 打印响应
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # 清理线程
    await thread.delete() if thread else None

await main()

# User: Plan me a day trip.

# TravelAgent: 我为您计划了一次美妙的日间旅行，目的地是埃及开罗。以下是您一天的行程安排：

### 上午
1. **吉萨金字塔群**  
   - 参观著名的金字塔，欣赏大金字塔和狮身人面像。这是世界七大奇迹之一，非常值得一看。

### 中午
2. **古埃及餐厅用餐**  
   - 在一家当地餐厅享用传统的埃及美食，比如法拉费尔、库斯库斯和塔比勒。

### 下午
3. **埃及博物馆**  
   - 探索丰富的古埃及历史，查看法老的遗物，包括图坦卡蒙的财宝和古代艺术品。

4. **老开罗区**  
   - 漫步于充满历史韵味的老开罗，参观古老的教堂和清真寺，感受千年文化的交融。

### 晚上
5. **尼罗河游船晚餐**  
   - 在尼罗河上享受一顿晚餐，欣赏河岸的美丽风景及夜晚的灯光表演。

希望您在开罗的日间旅行充满乐趣！如果您需要更多的细节或其他帮助，请告诉我。
